In [14]:
from os import listdir, rename
from os.path import isfile, join
from shutil import copyfile

import cv2
import numpy as np

import cPickle as pickle

import matplotlib.pyplot as plt
import matplotlib.image as mpimg
%matplotlib inline

from ball_detect_utils import hsv_to_bool_mask
from ball_detect_utils import hsv_to_circular_bool_mask

In [15]:
data = pickle.load(open("data.p", "rb"))

In [18]:
def jaccard(im_mask_pd, im_mask_gt):
    """
    - both input must be bool
    - return between 0 to 1, 1 the best
    """
    assert im_mask_pd.dtype == np.bool
    assert im_mask_gt.dtype == np.bool
    union = float(np.sum(np.logical_or(im_mask_pd, im_mask_gt)))
    interset = float(np.sum(np.logical_and(im_mask_pd, im_mask_gt)))
    if union == 0: 
        return 0.
    return interset / union

def hsv_to_jaccard(im_hsv, hsv_lows, hsv_highs,
                   im_mask_gt, enable_circular):
    if enable_circular:
        im_mask_pd = hsv_to_circular_bool_mask(im_hsv, 
                                               hsv_lows, hsv_highs)
    else:
        im_mask_pd = hsv_to_bool_mask(im_hsv, hsv_lows, hsv_highs)
    return jaccard(im_mask_pd, im_mask_gt)

def threshold_to_score(data,
                       hsv_lows, hsv_highs,
                       color,
                       enable_circular):
    jacards = []
    
    if color == 'orange':
        for key, d in data.iteritems():
            jacard = hsv_to_jaccard(d['im_hsv'], 
                                    orange_hsv_lows, orange_hsv_highs,
                                    d['im_mask_orange'], enable_circular)
            jacards.append(jacard)
    elif color == 'green':
        for key, d in data.iteritems():
            jacard = hsv_to_jaccard(d['im_hsv'], 
                                    orange_hsv_lows, orange_hsv_highs,
                                    d['im_mask_green'], enable_circular)
            jacards.append(jacard)
    else:
        raise
        
    return float(np.sum(jacards))

In [13]:
orange_hsv_lows = (0, 146, 120)
orange_hsv_highs = (16, 255, 255)
green_hsv_lows = (30, 80, 80)
green_hsv_highs = (50, 255, 255)
enable_circular = True

In [23]:
threshold_to_score(data,
                   orange_hsv_lows, orange_hsv_highs,
                   color='orange', enable_circular=True)

40.283758266072375